In [1]:
from functions import (prep, folding, inbu, LSTM_model)

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
np.random.seed(42)
source = "running-example"
flatten_by = 'Orders' # input("Enter the value for flatten_by (Orders, Items or Packages): ")
single_log = 'True' # input("Enter the value for single_log (True/False): ")
complete = 'False' # input("Enter the value for complete (True/False): ")

# Error handling for invalid input values
if flatten_by not in ['Orders', 'Items', 'Packages']:
    raise ValueError("Wrong Input: flatten_by must be one of ['Orders', 'Items', 'Packages']")

if single_log.lower() not in ['true', 'false', '1', '0']:
    raise ValueError("Wrong Input: single_log must be a boolean value (True/False)")

if complete.lower() not in ['true', 'false', '1', '0']:
    raise ValueError("Wrong Input: complete must be a boolean value (True/False)")

# Convert input values to boolean
single_log = single_log.lower() in ['true', '1']
complete = complete.lower() in ['true', '1'] 
if complete:
    csvname = flatten_by  + '_complete'
    fl = None
else:
    csvname = flatten_by  + '_filter'
    fl = prep.act_filter(flatten_by )

time_feat = ['Time_Diff', 'Time_Since_Start', 'Time_Since_Midnight','Weekday','Position']
other_features = ['Amount_Items','In_Package']
drops_col_order = ["weight", "price", "Event_ID", 'Products']
print("Settings:")
print(f"flatten_by: {flatten_by}")
print(f"single_log: {single_log}")
print(f"complete: {complete}")

Settings:
flatten_by: Orders
single_log: True
complete: False


In [2]:
## prep the ocel and readin
ocel, act_dict, cust_dict = prep.prepare_flat_ocel(source, flatten_on= flatten_by , filter= fl)
print(act_dict)
print(cust_dict)
## create the enriched and some more preprocessing as well as saving the single and enriched versions
ocel = prep.gen_enriched_single_plus_csv(OCEL = ocel,flatted_by = flatten_by ,csvname = csvname, drops_col= drops_col_order)
## adding features
ocel =prep.generate_features(ocel)
divisor = np.mean(ocel['Time_Diff'])  # average time between events
divisor2 = np.mean(ocel['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(ocel['Remaining_Time'])  # average time instance remaining
divisor3 = ocel.groupby('Case_ID')['Time_Since_Start'].apply(lambda x: (x.iloc[-1] - x).mean()).mean()

print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f"divisor3: {divisor3}")
print(len(ocel))

#folding the data 
ocel_train, ocel_test = folding.folding_train_test(ocel)

{'place order': 'A', 'confirm order': 'B', 'pay order': 'C', 'payment reminder': 'D'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}
divisor: 318986.03791832976
divisor2: 528541.2252072459
divisorTR: 679045.650752226
divisor3: 591967.807807143
6514


defining the features

In [3]:

act_feat = list(filter(lambda k: k.startswith('Act_') and not k.startswith('Next_Act_'), ocel_train.columns))
act_feat.remove('Act_!')
act_feat_dict = {index: value.replace('Act_', '') for index, value in enumerate(act_feat)}
target_act_feat = list(filter(lambda k: k.startswith('Next_Act_') and not k.startswith('Act_'), ocel_train.columns))
target_act_feat_dict = {index: value.replace('Next_Act_', '') for index, value in enumerate(target_act_feat)}

cust_feat = list(filter(lambda k: 'Cust_' in k, ocel_train.columns))

feature_select = act_feat + cust_feat * (single_log -1) + time_feat + other_features *(single_log -1)
print(f"Length of act_feat: {len(act_feat)}, Length of cust_feat: {len(cust_feat)}")

## define dimensions of inputs
max_trace_length = prep.gen_traces_and_maxlength_of_trace(ocel)[1]
target_act_length = len(target_act_feat)
number_of_train_cases = len(ocel_train)
num_of_features = len(feature_select)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}")



Length of act_feat: 4, Length of cust_feat: 18
Number of train cases: 4366, Max trace length: 9, Number of features: 9


get the inputs 

In [4]:
print(target_act_feat_dict)
print(act_feat_dict)

{0: '!', 1: 'A', 2: 'B', 3: 'C', 4: 'D'}
{0: 'A', 1: 'B', 2: 'C', 3: 'D'}


In [5]:
num_of_features

9

In [18]:
ocel

,Case_ID,Activity,Timestamp,Items,Customers,Packages,Amount_Items,Time_Diff,Time_Since_Start,Time_Since_Midnight,...,Cust_o,Cust_p,Cust_q,Next_Activity,Next_Act_!,Next_Act_A,Next_Act_B,Next_Act_C,Next_Act_D,Position
0,990001,A,2019-05-20 09:07:47,"[880001, 880004, 880003, 880002]",a,,4,0,0,32867,...,0,0,0,B,0.0,0.0,1.0,0.0,0.0,1
1,990001,B,2019-05-20 11:13:54,"[880001, 880004, 880003, 880002]",a,,4,7567,7567,40434,...,0,0,0,C,0.0,0.0,0.0,1.0,0.0,2
2,990001,C,2019-05-23 11:27:55,"[880001, 880004, 880003, 880002]",a,,4,260041,267608,41275,...,0,0,0,!,1.0,0.0,0.0,0.0,0.0,3
3,990002,A,2019-05-20 10:35:21,"[880008, 880005, 880006, 880007]",b,,4,0,0,38121,...,0,0,0,B,0.0,0.0,1.0,0.0,0.0,1
4,990002,B,2019-05-21 12:19:16,"[880008, 880005, 880006, 880007]",b,,4,92635,92635,44356,...,0,0,0,C,0.0,0.0,0.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6509,991999,B,2020-05-18 16:55:39,"[888154, 888157, 888155, 888158, 888156]",o,,5,15590,15590,60939,...,1,0,0,C,0.0,0.0,0.0,1.0,0.0,2
6510,991999,C,2020-05-29 17:20:28,"[888154, 888157, 888155, 888158, 888156]",o,,5,951889,967479,62428,...,1,0,0,!,1.0,0.0,0.0,0.0,0.0,3
6511,992000,A,2020-05-18 14:31:26,[888159],q,,1,0,0,52286,...,0,0,1,B,0.0,0.0,1.0,0.0,0.0,1
6512,992000,B,2020-05-18 14:49:03,[888159],q,,1,1057,1057,53343,...,0,0,1,C,0.0,0.0,0.0,1.0,0.0,2


In [6]:
X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(OCEL=ocel_train,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR, 
                                                                  single= single_log)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}")


Shape of X_train: (4366, 9, 9)
This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): (4366, 9, 9) => True
Shape of y_train_a: (4366, 5), this matches the desired shape (number_of_train_cases, target_act_length): (4366, 5) => True
Shape of y_train_t: (4366,), this matches the desired shape (number_of_train_cases, ): (4366,) => True
Shape of y_train_tr: (4366,), this matches the desired shape (number_of_train_cases, ): (4366,) => True


In [17]:
X_train[0]

array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       ],
       [1.       , 0.       , 0.       , 0.       , 1.       , 0.       

In [7]:

if single_log:
    model_file = csvname + '_single'
else:
    model_file = csvname + '_enriched'
# history = LSTM_model.LSTM_MODEL(X_train, y_train_a, y_train_t, y_train_tr,filename=model_file)
# print(history.history.keys())

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [21]:
from keras.models import load_model
from jellyfish import damerau_levenshtein_distance, levenshtein_distance
import pandas as pd
import distance

modelname = 'model_Orders_filter_single_128-1.30.h5'
model = load_model(f'./output_files/models/{modelname}')

X_test,y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(OCEL=ocel_test,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR, 
                                                                  single= single_log)

# y_t = y_t * divisor3

y = model.predict(X_test,verbose=1)
y_char = y[0][:][:]
y_t = y[1][:][:]
y_tr = y[2][:][:]
max_index_list = [np.argmax(pred) for pred in y_char]
pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
y_t = np.maximum(y_t, 0)
y_t1 = y_t * divisor
y_tr1 = y_tr * divisorTR

columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                  [col for col in ocel_test.columns if 'Cust_' in col] + \
                  ['Items', 'Customers', 'Packages', 'Next_Time_Since_Start',
                   'Next_Time_Since_Midnight', 'Next_Weekday', 'In_Package',
                   'Position', 'Time_Since_Midnight', 'Weekday', 'Amount_Items']

output_ocel = ocel_test.drop(columns=columns_to_drop).copy()
output_ocel['Pred_Activity'] = pred_act_list
output_ocel['Pred_Time_Diff'] = y_t1
output_ocel['Pred_Remaining_Time'] = y_tr1

output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

output_ocel
from sklearn import metrics


print(metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60)))
print(metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60)))
print(metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60),squared=False))
print(metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60),squared=False))

68/68 [==============================] - 1s 4ms/step
2.604857336903863
3.197816839635765
4.759801071662072
8.059703481044945


In [24]:
from jellyfish import damerau_levenshtein_distance, levenshtein_distance
import pandas as pd
import distance
from sklearn import metrics

# Generate inputs with varying prefix lengths
prefix_lengths = range(2, max_trace_length)  # List of prefix lengths to consider
results = []

for prefix_length in prefix_lengths:
    print(f"Results for Prefix Length {prefix_length}:")
    
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(OCEL=ocel_test,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR,
                                                                  single=single_log,
                                                                  prefix_length=prefix_length)

    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char = y[0][:][:]
    y_t = y[1][:][:]
    y_tr = y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t1 = y_t * divisor
    y_tr1 = y_tr * divisorTR

    columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                      [col for col in ocel_test.columns if 'Cust_' in col] + \
                      ['Items', 'Customers', 'Packages', 'Next_Time_Since_Start',
                       'Next_Time_Since_Midnight', 'Next_Weekday', 'In_Package',
                       'Position', 'Time_Since_Midnight', 'Weekday', 'Amount_Items']

    output_ocel = ocel_test.drop(columns=columns_to_drop).copy()
    output_ocel['Pred_Activity'] = pred_act_list
    output_ocel['Pred_Time_Diff'] = y_t1
    output_ocel['Pred_Remaining_Time'] = y_tr1

    output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
    output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
    output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
    output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

    mae_time_diff = metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60), output_ocel['Next_Time_Diff']/ (24 * 60 * 60)) 
    mae_remaining_time = metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60), output_ocel['Next_Remaining_Time']/ (24 * 60 * 60)) 
    rmse_time_diff = metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60), output_ocel['Next_Time_Diff']/ (24 * 60 * 60), squared=False) 
    rmse_remaining_time = metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60), output_ocel['Next_Remaining_Time']/ (24 * 60 * 60), squared=False) 

    # Store the results for the current prefix length
    results.append({
        'Prefix Length': prefix_length,
        'length': len(y_tr),
        'MAE Time Difference': mae_time_diff,
        'MAE Remaining Time': mae_remaining_time,
        'RMSE Time Difference': rmse_time_diff,
        'RMSE Remaining Time': rmse_remaining_time
    })

    # Output additional values based on Case_ID and prefix length
    for case_id in output_ocel['Case_ID'].unique():
        case_data = output_ocel[output_ocel['Case_ID'] == case_id]
        trace_length = len(case_data)
        if prefix_length <= trace_length:
            # print(f"\nAdditional values for Prefix Length {prefix_length} and Case ID {case_id}:")
            case_data_prefix = case_data[:prefix_length]
            #print(case_data_prefix.to_string(index=False))
            
            mae_time_diff_case = metrics.mean_absolute_error(case_data_prefix['Pred_Time_Diff']/ (24 * 60 * 60), case_data_prefix['Next_Time_Diff']/ (24 * 60 * 60))
            mae_remaining_time_case = metrics.mean_absolute_error(case_data_prefix['Pred_Remaining_Time']/ (24 * 60 * 60), case_data_prefix['Next_Remaining_Time']/ (24 * 60 * 60))
            rmse_time_diff_case = metrics.mean_squared_error(case_data_prefix['Pred_Time_Diff']/ (24 * 60 * 60), case_data_prefix['Next_Time_Diff']/ (24 * 60 * 60), squared=False)
            rmse_remaining_time_case = metrics.mean_squared_error(case_data_prefix['Pred_Remaining_Time']/ (24 * 60 * 60), case_data_prefix['Next_Remaining_Time']/ (24 * 60 * 60), squared=False)
            
            # print(f"\nMetrics for Prefix Length {prefix_length} and Case ID {case_id}:")
            # print("MAE Time Difference:", mae_time_diff_case)
            # print("MAE Remaining Time:", mae_remaining_time_case)
            # print("RMSE Time Difference:", rmse_time_diff_case)
            # print("RMSE Remaining Time:", rmse_remaining_time_case)

# Output the overall results
df_results = pd.DataFrame(results)
print("\nOverall Results:")
print(df_results)


Results for Prefix Length 2:
68/68 [==============================] - 0s 3ms/step
Results for Prefix Length 3:
68/68 [==============================] - 0s 2ms/step
Results for Prefix Length 4:
68/68 [==============================] - 0s 3ms/step
Results for Prefix Length 5:
68/68 [==============================] - 0s 3ms/step
Results for Prefix Length 6:
68/68 [==============================] - 0s 4ms/step
Results for Prefix Length 7:
68/68 [==============================] - 0s 4ms/step
Results for Prefix Length 8:
68/68 [==============================] - 0s 4ms/step

Overall Results:
   Prefix Length  length  MAE Time Difference  MAE Remaining Time   
0              2    2148             2.618566            3.179987  \
1              3    2148             2.609743            3.179557   
2              4    2148             2.609162            3.189315   
3              5    2148             2.606062            3.194762   
4              6    2148             2.605464            3.1965

In [ ]:
import csv
from datetime import timedelta
import distance
from jellyfish import damerau_levenshtein_distance
from keras.models import load_model
from numpy.compat import unicode
from sklearn import metrics

modelname = 'model_Orders_filter_single_128-1.30.h5'
model = load_model(f'./output_files/models/{modelname}')

predict_size = max_trace_length

# Get all unique activities in the DataFrame
chars = sorted(set(ocel['Activity'].unique()))
# Add '!' to the target_chars
target_chars = chars + ['!']
char_indices = dict((c, i) for i, c in enumerate(chars))
target_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i,c) for i, c in enumerate(target_chars))


# define helper functions
def encode(sentence, times, times3, maxlen=max_trace_length):
    num_features = len(chars) + 5
    X = np.zeros((1, max_trace_length, num_features), dtype=np.float32)
    leftpad = max_trace_length - len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t] - midnight
        for c in chars:
            if c == char:
                X[0, t + leftpad, char_indices[c]] = 1
        X[0, t + leftpad, len(chars)] = t + 1
        X[0, t + leftpad, len(chars) + 1] = times[t] / divisor
        X[0, t + leftpad, len(chars) + 2] = times2[t] / divisor2
        X[0, t + leftpad, len(chars) + 3] = timesincemidnight.seconds / 86400
        X[0, t + leftpad, len(chars) + 4] = times3[t].weekday() / 7
    return X


def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0
    for prediction in predictions:
        if prediction >= maxPrediction:
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol


one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []
flag = False
# Preallocate lists for storing results
results = []
with open('./output_files/results/suffix_and_remaining_time_%s.csv' % csvname, 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["CaseID", "Prefix length", "Ground truth", "Predicted", "Levenshtein", "Damerau", "Jaccard",
                         "Ground truth times", "Predicted times", "RMSE", "MAE"])

# Get relevant columns from the grouped data
group_columns = ['Case_ID', 'Activity', 'Time_Diff', 'Time_Since_Start', 'Timestamp']
grouped_data = ocel_test.groupby('Case_ID')[group_columns]

# Iterate over different prefix sizes
for prefix_size in range(2, max_trace_length):
    print(prefix_size)
    
    # Iterate over each case
    for case_id, group in grouped_data:
        if case_id == '990265':
            flag = True
            break
        line = group['Activity'].values
        Ptimes = group['Time_Diff'].values
        Ptimes2 = group['Time_Since_Start'].values
        Ptimes3 = pd.to_datetime(group['Timestamp']).dt.to_pydatetime().tolist()

        Ptimes = np.append(Ptimes, 0)
        cropped_line = ''.join(line[:prefix_size])
        cropped_times = Ptimes[:prefix_size]
        cropped_times3 = Ptimes3[:prefix_size]  # Convert numpy.datetime64 array to list

        if len(Ptimes2) < prefix_size:
            continue  # make no prediction for this case, since this case has ended already

        ground_truth = ''.join(line[prefix_size:prefix_size + predict_size])
        ground_truth_t = Ptimes2[prefix_size - 1]
        case_end_time = Ptimes2[-1]
        ground_truth_t = case_end_time - ground_truth_t

        predicted = ''
        total_predicted_time = 0

        for i in range(predict_size):
            enc = encode(cropped_line, cropped_times, cropped_times3)
            y = model.predict(enc, verbose=0)  # make predictions

            # split predictions into separate activity and time predictions
            y_char = y[0][0]
            y_t = y[1][0][0]
            prediction = getSymbol(y_char)  # undo one-hot encoding
            cropped_line += prediction

            y_t = np.maximum(y_t, 0)
            y_t1 = y_t * divisor
            cropped_times = np.append(cropped_times, y_t1)

            if prediction == '!':  # end of case was just predicted, therefore, stop predicting further into the future
                one_ahead_pred.append(total_predicted_time)
                one_ahead_gt.append(ground_truth_t)
                print('! predicted, end case')
                break

            y_t = y_t * divisor3
            next_time = cropped_times3[-1] + timedelta(seconds=y_t)
            cropped_times3.append(next_time)  # Append new datetime object
            total_predicted_time += y_t
            predicted += prediction

        output = []
        if len(ground_truth) > 0:
            output.append(case_id)
            output.append(prefix_size)
            output.append(str(ground_truth))
            output.append(str(predicted))
            output.append(1 - distance.nlevenshtein(predicted, ground_truth))
            dls = 1 - (damerau_levenshtein_distance(str(predicted), str(ground_truth)) / max(len(predicted), len(ground_truth)))
            dls = np.maximum(dls, 0)
            output.append(dls)
            output.append(1 - distance.jaccard(predicted, ground_truth))
            output.append(ground_truth_t)
            output.append(total_predicted_time)
            output.append('')
            output.append(metrics.mean_absolute_error([ground_truth_t], [total_predicted_time]))
            results.append(output)

    if flag:
        break
with open('./output_files/results/suffix_and_remaining_time_%s.csv' % csvname, 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # Write results to CSV
    spamwriter.writerows(results)

In [ ]:
# 990265 in 35
# in 36.3 new in 34

#         if flag:
#             break 
#         # Iterate over each case
#         for case_id, group in grouped_data:
#             if case_id == '990265':
#                 flag = True
#                 break